In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import time

# Chrome 옵션 설정
options = Options()
options.headless = False  # 화면을 보면서 디버깅하기 위해 headless 모드를 끔
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 스타일별 폴더 생성
styles = ['캐주얼']  # 예시로 '캐주얼'만 사용
base_url = 'https://www.musinsa.com/snap/main/recommend'
save_dir = 'musinsa_style_images'

def save_images(driver, style_dir, images, saved_images):
    for img_element in images:
        try:
            img_url = img_element.get_attribute('src')
            img_alt = img_element.get_attribute('alt')

            # 고유 파일명 생성
            img_name = f"{img_alt}.jpg"
            img_path = os.path.join(style_dir, img_name)

            if not os.path.exists(img_path):
                # 이미지 저장
                img_data = requests.get(img_url).content
                with open(img_path, 'wb') as handler:
                    handler.write(img_data)

                print(f"이미지 저장 완료: {img_name}")
                saved_images += 1

            if saved_images >= total_images_to_save:
                break

        except Exception as e:
            print(f"이미지 저장 중 오류 발생: {e}")
            continue
    
    return saved_images

try:
    for style in styles:
        # 스타일별 폴더 생성
        style_dir = os.path.join(save_dir, style)
        os.makedirs(style_dir, exist_ok=True)
        
        # 페이지 로드
        driver.get(base_url)
        
        # 스타일 필터 선택 및 클릭
        style_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@role="button" and @aria-haspopup="dialog" and contains(text(), "스타일")]'))
        )
        style_button.click()

        style_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//span[contains(text(), "{style}")]'))
        )
        style_option.click()

        # "스냅 보기" 버튼 클릭
        snap_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "스냅 보기")]'))
        )
        snap_button.click()
        
        # "인기순" 버튼 클릭
        popular_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "인기순")]'))
        )
        popular_button.click()

        # "최신순" 버튼 클릭
        latest_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "최신순")]'))
        )
        latest_button.click()

        # 페이지 로딩 대기
        time.sleep(8)  # 페이지 로딩을 기다리기 위해 8초 대기 (필요시 시간 조절)
        
        # 이미지 다운로드
        total_images_to_save = 60
        scroll_pause_time = 10  # 스크롤 후 대기 시간
        saved_images = 0

        while saved_images < total_images_to_save:
            images = driver.find_elements(By.XPATH, '//div[contains(@class, "relative")]//img')
            saved_images = save_images(driver, style_dir, images, saved_images)
            
            # 다음 이미지를 로드하기 위해 한 줄씩 스크롤 다운
            driver.execute_script("window.scrollBy(0, window.innerHeight / 3);")  # 한 줄씩 스크롤
            time.sleep(scroll_pause_time)  # 스크롤 후 잠시 대기
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if saved_images >= total_images_to_save or new_height == driver.execute_script("return window.pageYOffset + window.innerHeight"):
                break  # 페이지 끝에 도달했거나 이미지를 다 저장하면 종료

finally:
    # 브라우저 닫기
    driver.quit()


이미지 저장 완료: 내 프로필.jpg
이미지 저장 완료: snap_card_1269994053816281916.jpg
이미지 저장 완료: snap_card_1269994034487784664.jpg
이미지 저장 완료: snap_card_1269989546343082629.jpg
이미지 저장 완료: snap_card_1269988159512761417.jpg
이미지 저장 완료: snap_card_1269986873088501222.jpg
이미지 저장 완료: snap_card_1269984858639426255.jpg
이미지 저장 완료: snap_card_1269984185499772724.jpg
이미지 저장 완료: snap_card_1269983513366924810.jpg
이미지 저장 완료: snap_card_1269981617551689644.jpg
이미지 저장 완료: snap_card_1269981604933626170.jpg
이미지 저장 완료: snap_card_1269979407615961096.jpg
이미지 저장 완료: snap_card_1269978732107654021.jpg
이미지 저장 완료: snap_card_1269978634993099188.jpg
이미지 저장 완료: snap_card_1269978011337485695.jpg
이미지 저장 완료: snap_card_1269975331445664613.jpg
이미지 저장 완료: snap_card_1269974924588295560.jpg
이미지 저장 완료: snap_card_1269973993051818424.jpg
이미지 저장 완료: snap_card_1269973371439187135.jpg
이미지 저장 완료: snap_card_1269834093364649376.jpg
이미지 저장 완료: snap_card_1269833747437071751.jpg
이미지 저장 완료: snap_card_1269835834602692378.jpg
이미지 저장 완료: snap_card_1269835758518